In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd 
import json
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots                                                         
import plotly.io as pio
pio.renderers.default="iframe_connected"
from itertools import islice
import warnings
warnings.filterwarnings("ignore")

#  Transaction Data

In [ ]:
txn_df = pd.read_csv('/kaggle/input/transactions-fraud-datasets/transactions_data.csv')
txn_df

In [ ]:
txn_df.describe()

In [ ]:
txn_df.dtypes

In [ ]:
txn_df.date = pd.to_datetime(txn_df.date) # convert date to datetime
txn_df.amount = txn_df.amount.str.replace('$', '').astype(float)
txn_df.dtypes

## Check NULLs

In [ ]:
txn_df.isna().sum()

###   1. Handle NULL `merchant_state`

In [ ]:
nan_state = txn_df[txn_df.merchant_state.isna()]

In [ ]:
print(nan_state.shape)
nan_state.head()

In [ ]:
nan_state.merchant_city.unique()

In [ ]:
txn_df.merchant_state.fillna('ONLINE', inplace = True)
txn_df.isna().sum()

###   2. Handle NULL `zip`

In [ ]:
nan_zip = txn_df[txn_df.zip.isna()]
nan_zip.head()

In [ ]:
txn_df.loc[txn_df["merchant_city"] == "ONLINE", 'zip'] = 0 
txn_df.isna().sum()

In [ ]:
# Lookup I got, just to fill in values. Can skip if you want
missing_zip = {
  "Puerto Vallarta": "48300","Vatican City": "00120","Guadalajara": "44100","Santo Domingo": "10101","Montreal": "H3A","Toronto": "M5H","San Jose": "10101","Berlin": "10115","Mexico City": "01000","Shanghai": "200000","Cancun": "77500","Edinburgh": "EH1","Tallinn": "10111","Funafuti": "","Tapei": "100","Abu Dhabi": "00000","Beijing": "100000","Vilnius": "01100","Amsterdam": "1011","Tokyo": "100-0001","Athens": "10552","Ho Chi Minh City": "700000","Port au Prince": "6110","Dublin": "D01","Singapore": "018989","Paris": "75001","Johannesberg": "","Cabo San Lucas": "23450","Bangkok": "10200","Rome": "00184","Copenhagen": "1050","Kingston": "","Porto-Novo": "","London": "SW1A 1AA","Edmonton": "T5J","Calgary": "T2P","Brussels": "1000","Freetown": "","Jakarta": "10110","Bogota": "110111","Geneva": "1201","Vancouver": "V5K","Lisbon": "1100-148",
  "Wellington": "6011","Amman": "11118","Guatamala City": "01001","Hong Kong": "999077","Helsinki": "00100","Ulan Bator": "15160","Riyadh": "12611","Manila": "1000","Oslo": "0150","Budapest": "1051","Seoul": "04524","Jerusalem": "91000","Mumbai": "400001","Sydney": "2000","Monaco": "98000","Bucharest": "010011","Saint Petersburg": "190000","Vienna": "1010","Majuro": "96960","Delhi": "110001","Barcelona": "08001","Zurich": "8001","Oranjestad": "","Lima": "15001","Chisinau": "MD-2001","Acapulco": "39300","Dhaka": "1000","Madrid": "28001","Sao Paulo": "01000-000","Palikir": "96941","Tashkent": "100000","Santiago": "8320000",
  "Zagreb": "10000","Kolkata": "700001","Bangalore": "560001","Rio de Janeiro": "20000-000","Stockholm": "11120","Bandar Seri Begawan": "BA1111","Saint John's": "","Andorra La Vella": "AD500","Nairobi": "00100","Prague": "110 00","Nuku Alofa": "","Asmara": "","Honiara": "","Islamabad": "44000","Nassau": "","Rabat": "10000","Suva": "","Muscat": "113","Yaounde": "","Skopje": "1000","Tegucigalpa": "11101","Abuja": "900001","Belmopan": "","Montevideo": "11000","Istanbul": "34000","Manama": "","Moscow": "101000","Kuala Lumpur": "50000","Luxembourg": "L-1111","Algiers": "16000","Bridgetown": "BB11114",
  "Niamey": "","Cairo": "11511","Hanoi": "100000","Accra": "00233","Bratislava": "811 01","Kiev": "01001","Yaren District": "","Buenos Aires": "C1000","Juba": "","Podgorica": "81000","Colombo": "00100","Lahore": "54000","Reykjavik": "101","Ljubljana": "1000","Sanaa": "","Riga": "LV-1050","Doha": "00000","Karachi": "74000","Male": "20026","Ouagadougou": "","Harare": "","Praia": "","Malabo": "","Port Vila": "","Addis Ababa": "1000","Mbabane": "","Maputo": "1100","Warsaw": "00-001","Lusaka": "10101","Kingstown": "",
  "Tblisi": "0105","Baghdad": "10001","Apia": "","Monrovia": "1000","Bamako": "","Nicosia": "1010","Quito": "170101","Dakar": "","Libreville": "","Belgrade": "11000","Tunis": "1000","Pristina": "10000","Bishek": "720001","Victoria": "","Sarajevo": "71000","Panama City": "0819","Paramaribo": "","Yamoussoukro": "","Yangon": "11181","Tehran": "11369","Beirut": "1107","Valletta": "VLT 1117","Tirana": "1001","Conakry": "","Port Moresby": "111","Dili": "","Caracas": "1010","Khartoum": "11111","Port of Spain": "","Brazzaville": "","Baku": "AZ1000","Georgetown": "","Sao Paolo": "01000-000"
}


In [ ]:
txn_df["zip"] = txn_df["zip"].fillna(txn_df["merchant_city"].map(missing_zip))
txn_df.isna().sum()

###   3. Handle NULL `error`

In [ ]:
txn_df[txn_df.errors.isna()].fillna('No Errors', inplace = True)
txn_df.isna().sum()

The above didn't work because of `chained indexing.`

Pandas does not guarantee whether: `df[condition]` returns a view or a copy — so inplace=True becomes unreliable.

In [ ]:
txn_df['errors'].fillna('No Errors', inplace = True)
txn_df.isna().sum()

In [ ]:
txn_df.describe()

# Fraud Data

In [ ]:
with open('/kaggle/input/transactions-fraud-datasets/train_fraud_labels.json', 'r') as file:
    fraud = json.load(file)

In [ ]:
# list(fraud.items())[:3] # this gives me first 3 key-value paris, but everything is wrapped in target so we see full dict

list(fraud['target'].items())[:3] # this gives me first 3 key-value pairs

In [ ]:
fraud_df = pd.DataFrame(fraud['target'].items(), columns=['transaction_id', 'is_fraud'])
fraud_df.head(3)

In [ ]:
fraud_df.dtypes

In [ ]:
fraud_df.transaction_id = fraud_df.transaction_id.astype('int64')
print(fraud_df.shape)
fraud_df.dtypes

In [ ]:
frauds = pd.merge(txn_df,fraud_df,left_on='id',right_on='transaction_id',how='inner')
frauds.head(3)

# Users Data

In [ ]:
user_df = pd.read_csv('/kaggle/input/transactions-fraud-datasets/users_data.csv')
user_df

In [ ]:
user_df.dtypes

In [ ]:
user_df.per_capita_income = user_df.per_capita_income.astype('str').str.replace('$', '').astype('float')
user_df.yearly_income = user_df.yearly_income.astype('str').str.replace('$', '').astype('float')
user_df.total_debt = user_df.total_debt.astype('str').str.replace('$', '').astype('float')

user_df.dtypes

In [ ]:
user_df.isna().sum()

In [ ]:
user_df[user_df.id == 825]

In [ ]:
user_df_prefixed = user_df.rename(columns=lambda col: f"client_{col}")
user_cols_to_use = ["client_id","client_birth_year","client_birth_month","client_gender","client_latitude","client_longitude"]
user_df_relevant = user_df_prefixed[user_cols_to_use]
user_df_relevant.head()

In [ ]:
txn_user_df = frauds.merge(
    user_df_relevant,
    on="client_id",
    how="left"
)

txn_user_df.head()

In [ ]:
txn_user_df.isna().sum()

# Cards Data

In [ ]:
cards_df = pd.read_csv('/kaggle/input/transactions-fraud-datasets/cards_data.csv')
cards_df

In [ ]:
cards_df.dtypes

In [ ]:
cards_df.isna().sum()

In [ ]:
cards_df.describe()

In [ ]:
cards_df_prefixed = cards_df.rename(columns=lambda col: f"card_{col}" if col not in ["card_brand","card_type","card_number","client_id"] else col)
card_cols = [
    "card_id",
    "card_brand",
    "card_type",
    "card_has_chip",
    "card_acct_open_date"
]
cards_df_relevant = cards_df_prefixed[card_cols]
cards_df_relevant.head()

In [ ]:
pd.set_option('display.max_columns', 30)

In [ ]:
txn_user_card_df = txn_user_df.merge(
    cards_df_relevant,
    on="card_id",
    how="left"
)
txn_user_card_df.head()

In [ ]:
txn_user_card_df['is_refund'] = (txn_user_card_df['amount'] < 0).astype(int)
txn_user_card_df.head()

In [ ]:
txn_user_card_df.isna().sum()

In [ ]:
print(txn_user_card_df.shape)
txn_user_card_df.dtypes

# EDA on Fraud Data

In [ ]:
txn_user_card_df.isna().sum()

In [ ]:
txn_user_card_df.dtypes

### Keep main DF untouched, Make a copy & start EDA on that

In [ ]:
df = txn_user_card_df.copy()

In [ ]:
# Drop dup cols
df.drop(columns=['transaction_id'], inplace=True)

# Derive necessary cols
df['txn_hour'] = df['date'].dt.hour
df['txn_dayofweek'] = df['date'].dt.dayofweek
df['txn_month'] = df['date'].dt.month

df['client_age_at_txn'] = df['date'].dt.year - df['client_birth_year']

df['card_acct_open_date'] = pd.to_datetime(df['card_acct_open_date'])
df['card_age_days'] = (df['date'] - df['card_acct_open_date']).dt.days

df.head()

## Fraud %

In [ ]:
df.is_fraud.value_counts(normalize=True)

### INSIGHT -

Fraud Rate is **~0.15%** (that is ~1 fraud transaction out of every 670 transactions).

Also, this dataset is highly imbalanced, so accuracy becomes a bad metric. Instead, we use Recall.

Each row practically means - “Should we stop this transaction and potentially annoy a real customer?”

* False negatives = lost money
* False positives = angry customers

The whole project revolves around that tradeoff

In [ ]:
df.duplicated().sum()

## Question 1: Is fraud related to amount?

In [ ]:
df.groupby('is_fraud')['amount'].describe()

Negative amount (for refunds or whatever) stain the data, so we'll study purchases & refunds separately.

In [ ]:
purchases = df[df.amount > 0]
refunds   = df[df.amount < 0]

In [ ]:
purchases.groupby('is_fraud')['amount'].describe()

### INSIGHT - 
1. Fraud purchases are ~2.5× larger
2. Fraud has much higher variance
3. Fraud distribution is heavy-tailed

In [ ]:
refunds.groupby('is_fraud')['amount'].describe()

### INSIGHT - 
1. Fraud refunds are ~3× larger
2. Fraud refunds are rarer but extreme
3. Fraud resolution tends to involve high-value chargebacks

In [ ]:
df[df.is_fraud == 'Yes']['amount'].hist(bins=100)

## Question 2: Fraud vs transaction type

In [ ]:
df.groupby(['use_chip', 'is_fraud']).size().unstack().fillna(0)

In [ ]:
df.groupby('use_chip')['is_fraud'].value_counts(normalize=True).unstack()*100